In [ ]:
import ranking

import pandas as pd
import numpy as np
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer
from sentence_transformers import evaluation

import time

In [ ]:
SEED = 42
VAR_ST = (0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_val = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_val.append(df_val)
    lista_df_teste.append(df_teste)

# Usando os modelos ST

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import evaluation

In [ ]:
CAMINHO_MODELO = 'Dados/Resultados/ST/dist-v1/'

NOME_MODELO = (
               'distiluse-base-multilingual-cased-v1',
               'distiluse-base-multilingual-cased-v2',
               'paraphrase-multilingual-MiniLM-L12-v2',
               'paraphrase-multilingual-mpnet-base-v2'
              )

APELIDO_MODELO = (
                  'dbm-v1',
                  'dbm-v2',
                  'pml-v2',
                  'pmb-v2'
                 )

In [ ]:
def retornar_input_example(titulo_1, titulo_2, label):

    return(  InputExample(texts = [ titulo_1, titulo_2 ], label = label)  )

In [ ]:
def calcular_embedding(modelo, titulo):

    embedding = modelo.encode(titulo)

    return embedding

In [ ]:
def pipeline_st(df_treino, df_val, df_teste, nome_modelo):

    num_epocas = 3
    
    # carregando o modelo
    modelo = SentenceTransformer(nome_modelo)

    dados_treino = df_treino.apply(lambda row: retornar_input_example(row['titulo_1'], row['titulo_2'], float(row['match'])), axis = 1)
    dados_val = df_val.apply(lambda row: retornar_input_example(row['titulo_1'], row['titulo_2'], float(row['match'])), axis = 1)

    treino_dataloader = DataLoader(dados_treino, shuffle = True, batch_size = 1)
    #treino_perda = losses.CosineSimilarityLoss(model = modelo)
    treino_perda = losses.ContrastiveLoss(model = modelo)
    #avaliador = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dados_val)
    avaliador = evaluation.BinaryClassificationEvaluator.from_input_examples(dados_val)

    # fine-tune do modelo
    modelo.fit(
               train_objectives = [(treino_dataloader, treino_perda)],
               epochs = num_epocas,
               evaluator = avaliador
              )
    
    # calculando os embeddings
    embedding_1 = df_teste.apply(lambda row: calcular_embedding(modelo, row['titulo_1']), axis = 1)
    embedding_2 = df_teste.apply(lambda row: calcular_embedding(modelo, row['titulo_2']), axis = 1)

    similaridade = []
    for i in range(len(embedding_1)):
        # Calculando a matriz de similaridade. Quanto maior o score maior a similaridade.
        similaridade.append(np.inner(embedding_1[i], embedding_2[i]))

    return similaridade

In [ ]:
def calcular_y_pred(limite, similaridade):

    y_pred = [1 if num >= limite else 0 for num in similaridade]

    return y_pred


def montar_df_resultado(y_teste, y_pred, df_teste, nome):
    df_y = pd.DataFrame(
                        list(zip(
                                 y_teste, y_pred,
                                 df_teste["categoria"].to_list(),
                                 df_teste["titulo_1"].to_list(),
                                 df_teste["titulo_2"].to_list()
                                )
                       ), columns = ['match', 'pred', 'categoria', 'titulo_1', 'titulo_2'])

    return df_y


def salvar_similaridade(y_teste, similaridade, df_teste, nome, apelido):

    df_y = montar_df_resultado(y_teste, similaridade, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/ST-{apelido}/{nome}_similaridade.csv', index = False)


def salvar_y_pred(y_teste, y_pred, df_teste, nome, apelido, limite):

    df_y = montar_df_resultado(y_teste, y_pred, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/ST-{apelido}/ST_{limite}/Resultado/{nome}_y.csv', index = False)


def salvar_relatorio(y_teste, y_pred, nome, apelido, tempo, limite):

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome
    df_relatorio['tempo'] = tempo

    df_relatorio.to_csv(f'Dados/Resultados/ST-{apelido}/ST_{limite}/Relatório/{nome}_relatório.csv', index = True)

In [ ]:
flag_st = True
if flag_st == True:

    numero_modelo = 0
    nome_modelo = NOME_MODELO[numero_modelo]
    apelido_modelo = APELIDO_MODELO[numero_modelo]
    tam = 5

    lista_df_resultado = []
    for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

        y_teste = df_teste["match"].to_list()
        
        inicio_tempo = time.time()
        #y_teste = y_teste[:tam]
        #similaridade = pipeline_st(df_treino[:tam], df_val[:tam], df_teste[:tam], nome_modelo)
        similaridade = pipeline_st(df_treino, df_val, df_teste, nome_modelo)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        salvar_similaridade(y_teste, similaridade, df_teste, nome, apelido_modelo)

        for limite in VAR_ST:

            y_pred = calcular_y_pred(limite, similaridade)

            salvar_y_pred(y_teste, y_pred, df_teste, nome, apelido_modelo, ranking.remove_pontuacao(str(limite)) )
            salvar_relatorio(y_teste, y_pred, nome, apelido_modelo, tempo, ranking.remove_pontuacao(str(limite)) )
        
        '''y_pred = calcular_y_pred(similaridade, 0.5)

        report = classification_report(y_teste[:tam], y_pred, output_dict = True)
        #report = classification_report(y_teste, y_pred, output_dict = True)
        df_resultado = pd.DataFrame(report).transpose()
        df_resultado['modelo'] = nome
        df_resultado['tempo'] = tempo

        df_resultado.to_csv(f'Dados/Resultados/ST-{apelido_modelo}/{nome}_resultado.csv', index = True)

        df_y = pd.DataFrame(list(zip(y_teste[:tam], similaridade)), columns = ['match', 'pred'])
        df_y.to_csv(f'Dados/Resultados/ST-{apelido_modelo}/y/{nome}_y.csv', index = False)'''